In [ ]:
import pandas as pd
import json
import glob
import os
import numpy
from sklearn.linear_model import Lasso

In [ ]:
def load_data(path):

    res = {}
    for f in glob.glob(path):
        with open(f, "r") as fh:
            res[os.path.basename(f).replace(".json", "")] = pd.DataFrame(json.load(fh))

    df_prod = res["production_schedule"]
    df_prod["cu_pct"] = df_prod["chemistry"].apply(lambda x: x['cu_pct'])
    df_prod.drop("chemistry", axis=1, inplace=True)

    df_chem = res["previous_heats_with_properties"]
    df_chem["cu_pct"] = df_chem["chemistry"].apply(lambda x: x['cu_pct'])
    for b in ["bushling", "pig_iron", 'municipal_shred', 'skulls']:
        df_chem[b] = df_chem["actual_recipe"].apply(lambda x: x[b])
    
    df_chem["total_input"] = df_chem[["bushling", "pig_iron", 'municipal_shred', 'skulls']].sum(axis=1)
    for b in ["bushling", "pig_iron", 'municipal_shred', 'skulls']:
        df_chem[b] /= df_chem["total_input"]
        
    df_chem["yield"] = df_chem["tap_weight"] / df_chem["total_input"]
    
    df_chem.drop("chemistry", axis=1, inplace=True)
    df_chem.drop("actual_recipe", axis=1, inplace=True)

    df_order = res['scrap_orders']

    df_constr = res['constraints']
    df_constr = pd.DataFrame(map(lambda x: x[0], df_constr.values.tolist()))
    
    df_inven = res['scrap_inventory']
    
    return df_prod, df_chem, df_order, df_constr, df_inven

In [ ]:
df_prod, df_chem, df_order, df_constr, df_inven = load_data("data/1/*.json")
df_prod

In [ ]:
df_chem

# scrap quality estimation

In [ ]:
y = 1-df_chem["yield"].values
X = df_chem[["bushling", "pig_iron", "municipal_shred", "skulls"]].values

In [ ]:
trash_share = Lasso(alpha=0.00001,fit_intercept=False,precompute=True,positive=True)
trash_share.fit(X,y)

In [ ]:
yield_coef=1-trash_share.coef_
yield_coef

In [ ]:
pure_yield = df_chem[["bushling", "pig_iron", "municipal_shred", "skulls"]].values*yield_coef

In [ ]:
pure_yield

In [ ]:
y = df_chem["cu_pct"].values

In [ ]:
a2 = Lasso(alpha=0.00001,fit_intercept=False,precompute=True,positive=True)
a2.fit(pure_yield,y)

In [ ]:
copperport = a2.coef_ # cupper share

In [ ]:
X

In [ ]:
y

In [ ]:
df_order

In [ ]:
df_constr

In [ ]:
df_inven

In [ ]:
price_per_ton = numpy.zeros(4)
ind=0
for component in df_order['scrap_type'].drop_duplicates(keep='first').values:
    df = df_order[df_order['scrap_type'] == component][['weight', 'price_per_ton']]
    p = df['price_per_ton'].values
    w = df['weight'].values
    price_per_ton[ind] = sum(p * w) / sum(w)
    ind = ind+1
price_per_ton

In [ ]:
 inv_weight = df_inven['weight'].values

In [ ]:
results_df = pd.DataFrame({'Component':["bushling", "pig_iron", "municipal_shred", "skulls"],'cu_pct':copperport,'price_per_ton':price_per_ton,
                              'yield':yield_coef,'inv_weight':inv_weight})
results_df